In [2]:
import descarteslabs as dl
from datetime import datetime
from subprocess import check_output
import subprocess 
import json
import pickle
from datetime import timedelta

## Fetch fire pickle

In [30]:
def fetch_fire_status():
    # Synce fire pickle locally 
    try: 
        args = ['gsutil cp gs://dl-fires/imagery/fires.pickle temp/']
        p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(p.communicate())
    except:
        print "error, no fire status file"

## Read in latest Inciweb report

In [7]:
def read_latest_conus_file():
    # List all files 
    args = ['gsutil ls gs://dl-fires/inciweb-data/']
    p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    inciweb_data_files = p.communicate()[0]
 
    # Isolate latest report  
    latest_entry = inciweb_data_files.split('\ngs://dl-fires/inciweb-data/')[-1]
    conus_file = latest_entry.replace('\n', '')
    
    # Download and open the latest report     
    args = ['gsutil cp gs://dl-fires/inciweb-data/'+ conus_file +' temp/']
    p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print p.communicate()
    txt = open('temp/'+ conus_file)
    file_string_contents = txt.read()
    return file_string_contents

## Set up temp dir with necessary files

In [8]:
def pull_files_local():
    # Create local temp file 
    process = subprocess.call('mkdir temp', shell=True)
    # Sync fires status pickle and latest fire report local
    fetch_fire_status()
    file_string_contents = read_latest_conus_file()
    return file_string_contents


## Open File

In [9]:
# This could be refactored to include a class 
def initiate_fire_instance(fire_string): 
    fire_arr = fire_string.split()
    name_list = fire_arr[:-4]
    name = " ".join(name_list)
    fire_id = fire_arr[-4]
    lat = float(fire_arr[-3])
    lon = float(fire_arr[-2])
    coords=[lat,lon]
    acres = fire_arr[-1]
    check_fire_status(fire_id, name, coords, acres)

In [10]:
def iterate_over_file(string):
    counter = 0
    for line in iter(string.splitlines()):
        if counter == 0:
            counter +=1
        else: 
            initiate_fire_instance(line)
            counter +=1

## Read in fire instances

In [11]:
def get_fire_instances(): 
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b

## Check if fire instance exists 

In [12]:
def check_fire_status(fire_id, name, coordinates, acres):
    fires = get_fire_instances() 
    print fires
    if fire_id in fires:
        try:
            last_checked =  fires[fire_id]['last_checked']
        except KeyError:
            last_checked = fires[fire_id]['date_entered']
        existing_fire(name, fire_id, coordinates, last_checked)
    else:
        print "New fire"
        new_fire_instance(fire_id, name, coordinates, acres)
        

## Logic for existing fire instance or new fire instance

In [13]:
def new_fire_instance(fire_id, name, coordinates, acres): 
    five_days_ago = str(datetime.today() + timedelta(days=-5))
    find_imagery(name, five_days_ago ,coordinates, fire_id) 
    
    # Make bucket for imagery 
    'gs://dl-fires/imagery'+ fire_id
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
        b[fire_id] = {
            'name': 'Mallard Fire',
            'date_entered': datetime.today().isoformat()
        }
        
    with open('temp/fires.pickle', 'wb') as handle:
        pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [14]:
def existing_fire(name, fire_id, coordinates, last_checked):
#     bucket = 'gs://dl-fires/imagery/'+ fire_id
    find_imagery(name, last_checked ,coordinates, fire_id ) 
    with open('temp/fires.pickle', 'rb') as handle:
        b = pickle.load(handle)
        b[fire_id]['last_checked'] = datetime.today().isoformat()
        
    with open('temp/fires.pickle', 'wb') as handle:
        pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)


## Find imagery

In [15]:
def make_tile(coords): 
    # to get the tile centered around your lat/lon, give it a tilesize of 1 and a pad of the half-width of your desired image (in meters)
    print(coords)
    bounding_box = dl.raster.dltile_from_latlon(
        lat=coords[0],
        lon=coords[1], 
        resolution=30, 
        tilesize=2, 
        pad=600)

    return bounding_box
    

In [16]:
def search_products(date, geometry):
    print(date)
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    print("The number of total images found is {}.".format(len(images['features'])))
    return [image['id'] for image in images['features']]

In [17]:
def download_imagery(ids, geometry, fire_id):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['swir2', 'swir1', 'nir', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
                        cutline=geometry,
                        save= True,
                        outfile_basename="temp/"+image_id)

        # Sync images to associated bucket
        args = ['gsutil cp  temp/'+image_id+ '.tif gs://dl-fires/imagery/'+fire_id+'/']
        p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        

In [18]:
def find_imagery(fire_name, date, coords, fire_id, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
    # if footprint:
        # image_ids = search_products(date,footprint)
        # download_imagery(image_ids, footprint)
    # else:
    dl_tile = make_tile(coords)
    image_ids = search_products(date, dl_tile)
    download_imagery(image_ids, dl_tile, fire_id)

## Run script

In [20]:
def clean_up():
    # Write fire status pickle file back to cloud
    print("Writing pickle file to g cloud")
    args = ['gsutil cp temp/fires.pickle gs://dl-fires/imagery/']
    p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(p.communicate())
    
    # remove temp files 
    print("Deleting local temp files, final step.")
    subprocess.call('rm -rf temp/', shell=True)

In [26]:
def run_script():
    latest_fires = pull_files_local()
    iterate_over_file(latest_fires)
    clean_up()

In [31]:
run_script()

('', 'Copying gs://dl-fires/imagery/fires.pickle...\n/ [0 files][    0.0 B/  3.2 KiB]                                                \r/ [1 files][  3.2 KiB/  3.2 KiB]                                                \r\nOperation completed over 1 objects/3.2 KiB.                                      \n')
('', 'Copying gs://dl-fires/inciweb-data/conus_fires_20180521-00UTC.txt...\n/ [0 files][    0.0 B/  1.0 KiB]                                                \r/ [1 files][  1.0 KiB/  1.0 KiB]                                                \r\nOperation completed over 1 objects/1.0 KiB.                                      \n')
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:31.109635'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-1

The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:31.109635'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:43.494095'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:38:20.877337'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:10.184837'}, 'id:5761': {'date_entered': '2018-05-18T15:56:38.113888',

{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:31.109635'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:43.494095'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:38:20.877337'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:10.184837'}, 'id:5761': {'date_entered': '2018-05-18T15:56:38.113888', 'name': 'Mallard Fire', 'last_checked'

2018-05-19T22:39:14.921081
The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:31.109635'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:43.494095'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2

The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:06.670746'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:43.494095'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2018-05-18T15:56:38.113888',

2018-05-19T22:39:43.090238
The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:06.670746'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:39:43.494095'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2

The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:06.670746'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.749924'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:11.986916'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:07.667149'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-19T22:40:06.946345'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2018-05-18T15:56:38.113888',

The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:06.670746'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.203570'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.625881'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:07.667149'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.406275'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2018-05-18T15:56:38.113888',

2018-05-19T22:40:12.561879
The number of total images found is 0.
{'id:5744': {'date_entered': '2018-05-18T15:56:49.467602', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:06.670746'}, 'id:5746': {'date_entered': '2018-05-18T15:59:02.780882', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.203570'}, 'id:5747': {'date_entered': '2018-05-18T16:00:00.427086', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.625881'}, 'id:5740': {'date_entered': '2018-05-18T15:58:31.553089', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:07.667149'}, 'id:5741': {'date_entered': '2018-05-18T16:00:00.212577', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.554327'}, 'id:5742': {'date_entered': '2018-05-18T15:59:34.533723', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:08.406275'}, 'id:5763': {'date_entered': '2018-05-18T15:56:19.391119', 'name': 'Mallard Fire', 'last_checked': '2018-05-20T21:14:05.732905'}, 'id:5761': {'date_entered': '2